In [2]:
import pandas as pd
import numpy as np
import random as rnd
import os
from sklearn import preprocessing


In [3]:
###Fill missing values with mean/mode

from sklearn.base import TransformerMixin

class meanModeImputer(TransformerMixin):

    def __init__(self):
        pass
    
 
    def fit(self, data, y=None):

        self.fill = pd.Series([data[c].value_counts().index[0]
            if data[c].dtype == np.dtype('O') else data[c].mean() for c in data],
            index=data.columns)

        return self

    def transform(self, data, y=None):
        return data.fillna(self.fill)

In [4]:
# GKnn Algorithm
from numpy import mean
from statistics import mode
import time

def GKnn(impute_df):
    
    for i in range(len(empty_loc_cols)-1):

        ## Differentiate numerical and categorical data
        numerical_cols = impute_df._get_numeric_data().columns #fetches just the numeric columns
        cols = impute_df.columns                                 # entire columns in dataset
        categorical_cols = list(set(cols) - set(numerical_cols)) # gives us just the categorical columns
        num_df = impute_df.loc[:,numerical_cols]                 #locates all rows in numerical dataframe column by their labels
        cat_df = impute_df.loc[:,categorical_cols]               #locates all columns in categorical dataframe column by their labels
        
        
        num_ck=num_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of emptyelements in a numeric DF
        maxIter = 2
        num_cp=num_df.copy()
        num_cp.drop([i + 1], axis = 0)                           #dropping adjacent emptyrows on indexing for numerical dataset to avoid duplication

        cat_ck=cat_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of empty elements in a categorical DF
        cat_cp=cat_df.copy()
        cat_cp.drop([i + 1], axis = 0)                           # dropping adjacent emptyrows on indexing for categorical dataset to avoid duplictaion
        
        
        def greyRelationalGrade(impute_df):
           
            t1=pd.DataFrame() #initializing a dataframe
            t2=pd.DataFrame(data = None, columns=cat_cp.columns, index=cat_cp.index) #has the shape of the cat_cp dataframe
            
            for j in range(num_cp.index.size):
                temp=pd.Series(num_cp.iloc[j,:]-num_ck)
                t1=t1.append(temp,ignore_index=True)
                
                if(cat_cp.iloc[j,:].equals(cat_ck)):
                    t2.iloc[j,:] = 1
                else:
                    t2.iloc[j,:] = 0

            t1.reset_index(drop=True, inplace=True)
            t2.reset_index(drop=True, inplace=True)
            t = pd.concat([t1, t2], axis=1)

                
            mmax=t.abs().max().max()
            mmin=t.abs().min().min()
            rho=0.5

            # Computing grey correlation coefficient
            GRC=((mmin+rho*mmax)/(abs(t)+rho*mmax))

            # Computing the GRG and getting the rank
            GRG=GRC.sum(axis=1)/GRC.columns.size
            GRGSort = GRG.sort_values(ascending= False)
            
            return GRGSort
        
 
        count = 0
        prev = pd.DataFrame(data=incomplete_df.iloc[empty_loc_cols]).mean()
        for j in range(len(empty_loc[0])-1):
            result= greyRelationalGrade(impute_df)
            resultSorted = result.sort_values(ascending = False)
            neighbours =  resultSorted.iloc[0:5].index
            # find the mean , and most frequent of the values using the index of these instances
            values  = impute_df.iloc[neighbours]
            num_neigh= values._get_numeric_data().columns
            cat_neigh = list(set(values) - set(num_neigh))
            cat_neigh_cols = values.loc[:, cat_neigh]
            num_neigh_cols = values.loc[:,num_neigh]
            nMean = num_neigh_cols.mean(axis = 0)
            
            ## imputation using k nearest neighbor instances
            if(empty_loc_cols[i] == empty_loc[0][j]):
                impute_df.iloc[empty_loc[0][j], empty_loc[1][j]] = nMean[len(empty_loc)-1]
            count =+ 1
            tol = 10e-4   
            
            ##convergence criteria
            if count >= maxIter or max(abs(impute_df.iloc[empty_loc_cols].mean() - prev )) < tol:
                break
            prev = impute_df.iloc[empty_loc_cols].mean()
    
    return impute_df


In [5]:
#Compute NRMS

from sklearn import metrics
#Frobenius Function
def frob(x):
    return np.linalg.norm(x, 'fro')

#NRMS
def nrms(original, imputed):
    x = frob(imputed - original) / frob(original)
    return x

In [6]:
## import xlsx files as pandas dataframe
pathIncomplete = '/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Incomplete Datasets Without Labels/Iris'
complete_df = pd.read_excel('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Original Datasets Without Labels/Iris.xlsx', header = None)
incomplete_df_name_list = []
incomplete_df_list = []
for filename in os.listdir(pathIncomplete):
    if filename.endswith('.xlsx'):
        incomplete_df_list.append(pd.read_excel(pathIncomplete+"/"+filename, header=None))
        incomplete_df_name_list.append(filename.split(".")[0])

In [7]:
os.listdir(pathIncomplete)

['Iris_NG_10.xlsx',
 'Iris_NE_5.xlsx',
 'Iris_NN_10.xlsx',
 'Iris_C_5.xlsx',
 'Iris_AG_1.xlsx',
 '.DS_Store',
 'Iris_NE_20.xlsx',
 'Iris_AE_1.xlsx',
 'Iris_NG_5.xlsx',
 'Iris_NL_20.xlsx',
 'Iris_AW_10.xlsx',
 'Iris_AL_5.xlsx',
 'Iris_NG_20.xlsx',
 'Iris_NN_1.xlsx',
 'Iris_NN_20.xlsx',
 'Iris_NW_5.xlsx',
 'Iris_NL_1.xlsx',
 'Iris_NE_10.xlsx',
 'Iris_AN_5.xlsx',
 'Iris_AW_20.xlsx',
 'Iris_NL_10.xlsx',
 'Iris_AW_1.xlsx',
 'Iris_AG_5.xlsx',
 'Iris_AG_20.xlsx',
 'Iris_C_1.xlsx',
 'Iris_NE_1.xlsx',
 'Iris_AN_20.xlsx',
 'Iris_AE_10.xlsx',
 'Iris_NG_1.xlsx',
 'Iris_NW_20.xlsx',
 'Iris_AL_10.xlsx',
 'Iris_AE_5.xlsx',
 'Iris_C_20.xlsx',
 'Iris_NW_1.xlsx',
 'Iris_AG_10.xlsx',
 'Iris_NN_5.xlsx',
 'Iris_AN_10.xlsx',
 'Iris_AL_1.xlsx',
 'Iris_AW_5.xlsx',
 'Iris_AN_1.xlsx',
 'Iris_AE_20.xlsx',
 'Iris_C_10.xlsx',
 'Iris_AL_20.xlsx',
 'Iris_NW_10.xlsx',
 'Iris_NL_5.xlsx']

In [8]:
imputed_df_list = []
nrms_data = []
index = 0
for incomplete_df in incomplete_df_list:

    incomplete_df.reset_index()
    complete_df.reset_index()
    
    complete_df.columns=["Col"+str(i) for i in range(complete_df.shape[1])] # gives an arrary size of column count
    incomplete_df.columns=["Col"+str(i) for i in range(incomplete_df.shape[1])] # gives array size of column count(incompleteDF)
    
    ## find empty locations
    empty_loc = np.where(pd.isnull(incomplete_df))
    empty_loc_cols = np.unique(empty_loc[0])
    empty_loc_col = np.unique(empty_loc[1])
    
   

    ## Replace NaN values to mean values
    incomplete_df = meanModeImputer().fit_transform(incomplete_df)
    
    ## GKnn
    imputed_df = GKnn(incomplete_df)
    
    #Compute NRMS
    nrmser = nrms(complete_df, imputed_df)
    nrms_data.append({incomplete_df_name_list[index]: nrmser})
    
    imputed_df_list.append(imputed_df)
    index+=1
    
nrms_data


[{'Iris_NG_10': 0.1116453899318624},
 {'Iris_NE_5': 0.051010925825812736},
 {'Iris_NN_10': 0.10815080446155419},
 {'Iris_C_5': 0.06965231696668366},
 {'Iris_AG_1': 0.034925985541628346},
 {'Iris_NE_20': 0.08325386436393632},
 {'Iris_AE_1': 0.036494706917711034},
 {'Iris_NG_5': 0.08538114383560841},
 {'Iris_NL_20': 0.11334840792558903},
 {'Iris_AW_10': 0.07578781482174818},
 {'Iris_AL_5': 0.056372550013124224},
 {'Iris_NG_20': 0.12426960717246611},
 {'Iris_NN_1': 0.02737557770292064},
 {'Iris_NN_20': 0.1429579779890016},
 {'Iris_NW_5': 0.07536950858807578},
 {'Iris_NL_1': 0.013948634686194113},
 {'Iris_NE_10': 0.09433779459930168},
 {'Iris_AN_5': 0.05871999700453664},
 {'Iris_AW_20': 0.10852712076256835},
 {'Iris_NL_10': 0.08218287933399837},
 {'Iris_AW_1': 0.03722834342343405},
 {'Iris_AG_5': 0.05325663248556093},
 {'Iris_AG_20': 0.09876116898060147},
 {'Iris_C_1': 0.029848545777140367},
 {'Iris_NE_1': 0.012919221153847827},
 {'Iris_AN_20': 0.10713064939281934},
 {'Iris_AE_10': 0.07813

In [9]:
#save imputed dataframes to xlsx
index = 0
for df in imputed_df_list:
    df.to_excel("/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/Iris_K5/"+incomplete_df_name_list[index]+".xlsx")
    index+=1


In [10]:
# Make Dictionary of dataset_name : nrms_values
nrms_dict = {}
index = 0
for imputed_df in imputed_df_list:
    nrmser = nrms(complete_df, imputed_df)
    nrms_dict.update( {incomplete_df_name_list[index]: nrmser} )
    index+=1
nrms_dict

{'Iris_NG_10': 0.1116453899318624,
 'Iris_NE_5': 0.051010925825812736,
 'Iris_NN_10': 0.10815080446155419,
 'Iris_C_5': 0.06965231696668366,
 'Iris_AG_1': 0.034925985541628346,
 'Iris_NE_20': 0.08325386436393632,
 'Iris_AE_1': 0.036494706917711034,
 'Iris_NG_5': 0.08538114383560841,
 'Iris_NL_20': 0.11334840792558903,
 'Iris_AW_10': 0.07578781482174818,
 'Iris_AL_5': 0.056372550013124224,
 'Iris_NG_20': 0.12426960717246611,
 'Iris_NN_1': 0.02737557770292064,
 'Iris_NN_20': 0.1429579779890016,
 'Iris_NW_5': 0.07536950858807578,
 'Iris_NL_1': 0.013948634686194113,
 'Iris_NE_10': 0.09433779459930168,
 'Iris_AN_5': 0.05871999700453664,
 'Iris_AW_20': 0.10852712076256835,
 'Iris_NL_10': 0.08218287933399837,
 'Iris_AW_1': 0.03722834342343405,
 'Iris_AG_5': 0.05325663248556093,
 'Iris_AG_20': 0.09876116898060147,
 'Iris_C_1': 0.029848545777140367,
 'Iris_NE_1': 0.012919221153847827,
 'Iris_AN_20': 0.10713064939281934,
 'Iris_AE_10': 0.07813293846991898,
 'Iris_NG_1': 0.040768764712204295,
 'I

In [11]:
# Download Dictionary as csv file
import csv
with open('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/NRMS_AE/NRMS/Iris_k5.csv', 'w') as csv_file:  
    writeTo = csv.writer(csv_file)
    for key, value in nrms_dict.items():
        writeTo.writerow([key, value])

In [12]:
nrms_data

[{'Iris_NG_10': 0.1116453899318624},
 {'Iris_NE_5': 0.051010925825812736},
 {'Iris_NN_10': 0.10815080446155419},
 {'Iris_C_5': 0.06965231696668366},
 {'Iris_AG_1': 0.034925985541628346},
 {'Iris_NE_20': 0.08325386436393632},
 {'Iris_AE_1': 0.036494706917711034},
 {'Iris_NG_5': 0.08538114383560841},
 {'Iris_NL_20': 0.11334840792558903},
 {'Iris_AW_10': 0.07578781482174818},
 {'Iris_AL_5': 0.056372550013124224},
 {'Iris_NG_20': 0.12426960717246611},
 {'Iris_NN_1': 0.02737557770292064},
 {'Iris_NN_20': 0.1429579779890016},
 {'Iris_NW_5': 0.07536950858807578},
 {'Iris_NL_1': 0.013948634686194113},
 {'Iris_NE_10': 0.09433779459930168},
 {'Iris_AN_5': 0.05871999700453664},
 {'Iris_AW_20': 0.10852712076256835},
 {'Iris_NL_10': 0.08218287933399837},
 {'Iris_AW_1': 0.03722834342343405},
 {'Iris_AG_5': 0.05325663248556093},
 {'Iris_AG_20': 0.09876116898060147},
 {'Iris_C_1': 0.029848545777140367},
 {'Iris_NE_1': 0.012919221153847827},
 {'Iris_AN_20': 0.10713064939281934},
 {'Iris_AE_10': 0.07813

In [13]:
imputed_df_list

[         Col0  Col1      Col2      Col3
 0    3.440000   3.5  1.400000  0.200000
 1    4.900000   3.0  1.400000  1.223256
 2    4.700000   3.2  1.300000  0.200000
 3    5.860465   3.1  1.500000  0.200000
 4    5.000000   3.6  3.772794  0.200000
 ..        ...   ...       ...       ...
 145  6.700000   3.0  5.200000  2.300000
 146  6.300000   2.5  5.000000  1.900000
 147  6.500000   3.0  5.200000  2.000000
 148  6.200000   3.4  5.400000  1.223256
 149  5.900000   3.0  5.100000  1.800000
 
 [150 rows x 4 columns],
      Col0     Col1  Col2      Col3
 0     5.1  3.50000   1.4  0.200000
 1     4.9  3.00000   1.4  0.200000
 2     4.7  3.20000   1.3  0.200000
 3     4.6  3.10000   1.5  0.200000
 4     5.0  3.10979   1.4  0.200000
 ..    ...      ...   ...       ...
 145   6.7  3.00000   5.2  2.300000
 146   6.3  2.50000   5.0  1.900000
 147   6.5  3.00000   5.2  2.000000
 148   6.2  3.40000   5.4  1.213986
 149   5.9  3.00000   5.1  1.800000
 
 [150 rows x 4 columns],
          Col0  Col1  